# Homework and bake-off: pragmatic color descriptions

In [ ]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Summer 2021"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [All two-word examples as a dev corpus](#All-two-word-examples-as-a-dev-corpus)
1. [Dev dataset](#Dev-dataset)
1. [Random train–test split for development](#Random-train–test-split-for-development)
1. [Question 1: Improve the tokenizer [1 point]](#Question-1:-Improve-the-tokenizer-[1-point])
1. [Use the tokenizer](#Use-the-tokenizer)
1. [Question 2: Improve the color representations [1 point]](#Question-2:-Improve-the-color-representations-[1-point])
1. [Use the color representer](#Use-the-color-representer)
1. [Initial model](#Initial-model)
1. [Question 3: GloVe embeddings [1 point]](#Question-3:-GloVe-embeddings-[1-point])
1. [Try the GloVe representations](#Try-the-GloVe-representations)
1. [Question 4: Color context [3 points]](#Question-4:-Color-context-[3-points])
1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bakeoff [1 point]](#Bakeoff-[1-point])
1. [Submission Instruction](#Submission-Instruction)

## Overview

This homework and associated bake-off are oriented toward building an effective system for generating color descriptions that are pragmatic in the sense that they would help a reader/listener figure out which color was being referred to in a shared context consisting of a target color (whose identity is known only to the describer/speaker) and a set of distractors.

The notebook [colors_overview.ipynb](colors_overview.ipynb) should be studied before work on this homework begins. That notebook provides backgroud on the task, the dataset, and the modeling code that you will be using and adapting.

The homework questions are more open-ended than previous ones have been. Rather than asking you to implement pre-defined functionality, they ask you to try to improve baseline components of the full system in ways that you find to be effective. As usual, this culminates in a prompt asking you to develop a novel system for entry into the bake-off. In this case, though, the work you do for the homework will likely be directly incorporated into that system (not required, but an efficient way to work at the very least).

## Set-up

See [colors_overview.ipynb](colors_overview.ipynb) for set-up in instructions and other background details.

In [1]:
from colors import ColorsCorpusReader
from nltk.translate.bleu_score import corpus_bleu
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from torch_color_describer import ContextualColorDescriber
from torch_color_describer import create_example_dataset
import matplotlib.pyplot as plt
import utils
from utils import START_SYMBOL, END_SYMBOL, UNK_SYMBOL

In [2]:
utils.fix_random_seeds()

In [3]:
COLORS_SRC_FILENAME = os.path.join(
    "data", "colors", "filteredCorpus.csv")

## All two-word examples as a dev corpus

So that you don't have to sit through excessively long training runs during development, I suggest working with the two-word-only subset of the corpus until you enter into the late stages of system testing.

In [4]:
dev_corpus = ColorsCorpusReader(
    COLORS_SRC_FILENAME,
    word_count=2,
    normalize_colors=True)

In [5]:
dev_examples = list(dev_corpus.read())

This subset has about one-third the examples of the full corpus:

In [6]:
dev_examples[0].colors

[[0.8388888888888889, 0.5, 0.86],
 [0.8083333333333333, 0.5, 0.59],
 [0.8361111111111111, 0.5, 0.57]]

### Content length distribution over total dataset

#### 1-word = 40% <br></br> 2-word = 25% <br></br> 3+ words = 35%

## Dev dataset

The first step is to extract the raw color and raw texts from the corpus:

In [50]:
dev_rawcols, dev_texts = zip(*[[ex.colors, ex.contents] for ex in dev_examples])
len(dev_rawcols), len(dev_texts)

(13890, 13890)

In [51]:
lowers = [re.findall('\w+', text.lower()) for text in dev_texts]

In [52]:
phrases = [' '.join(words) for words in lowers]
df = pd.DataFrame(phrases, columns=['words'])

In [112]:
def find(df: pd.DataFrame, word:str) -> pd.Series:
    return df[df['words'].str.contains(word)]

def changer(orig_df: pd.DataFrame, frm:str, to:str) -> pd.DataFrame:
    new = orig_df.copy()
    series = find(new, frm)
    inds = list(series.index)
    new.loc[inds] = new.loc[inds].apply(lambda x: x.str.replace(frm, to))
    return new
def single(ind, frm, to='green'):
    test.loc[ind] = test.loc[ind].str.replace(frm, to)

In [58]:
test = changer(df, 'gren', 'green')

In [59]:
test = changer(test, 'grn', 'green')

In [63]:
test = changer(test, 'gray', 'grey')

In [132]:
single(12123,  'greep', 'green')

In [188]:
single(12199, 'brigther', 'brighter')

In [189]:
single(13185, 'brigth', 'bright')

In [197]:
single(6487, 'brite', 'brighter')

In [137]:
single(10855, 'greyihs', 'greyish')

In [143]:
single(11268, 'greylish', 'greyish')

In [144]:
single(11270, 'greylish', 'greyish')

In [146]:
test = changer(test, 'greysih', 'greyish')

In [150]:
test = changer(test, 'greyist', 'greyest')

In [155]:
test = changer(test, 'bight', 'bright')

In [157]:
test = changer(test, 'birght', 'bright')

In [158]:
test = changer(test, 'birhgt', 'bright')

In [160]:
test = changer(test, 'birte', 'bright')

In [165]:
test = changer(test,  'brigher', 'brighter')

In [169]:
test = changer(test, 'brighest', 'brightest')

In [172]:
test = changer(test, 'brighet', 'bright')

In [175]:
test = changer(test,  'bright5', 'bright')

In [178]:
test = changer(test, 'brighness', 'brightness')

In [181]:
test = changer(test, 'brightet', 'brightest')

In [184]:
test = changer(test, 'brightp', 'bright')

In [186]:
test = changer(test,  'brighttttt', 'bright')

In [193]:
test = changer(test, 'brihgt', 'bright')

In [195]:
test = changer(test, 'briht', 'bright')

In [199]:
test = changer(test, 'brite', 'bright')

In [207]:
test = changer(test, 'brighterr', 'brighter')

In [266]:
single(6753, 'prple', 'purple')

In [265]:
single(6681, 'prpl', 'purple')

In [264]:
single(4672, 'prp', 'purple')

In [212]:
test=changer(test, 'purpple', 'purple')

In [214]:
test = changer(test, 'purpp', 'purple')

In [216]:
test = changer(test, 'purpole', 'purple')

In [218]:
test = changer(test, 'purply', 'purple')

In [221]:
test = changer(test, 'purplr', 'purple')

In [223]:
test = changer(test,  'purpleyellowish', 'purple yellowish')

In [225]:
test = changer(test,  'purpley', 'purple')

In [229]:
test = changer(test,  'purplepurple', 'purple')

In [231]:
test = changer(test,  'purplelish', 'purpleish')

In [234]:
test = changer(test, 'purpile', 'purple')

In [236]:
test = changer(test, 'purpil', 'purple')

In [238]:
test = changer(test, 'purpel', 'purple')

In [240]:
test = changer(test, 'purlplish', 'purpleish')

In [242]:
test = changer(test,  'purle', 'purple')

In [245]:
test = changer(test,  'puprle', 'purple')

In [247]:
test = changer(test, 'puplish', 'purpleish')

In [249]:
test = changer(test,  'pupleish', 'purpleish')

In [251]:
test = changer(test, 'puple', 'purple')

In [256]:
test = changer(test,  'grreen', 'green')

In [258]:
test = changer(test, 'grrenish', 'greenish')

In [260]:
test = changer(test, 'grteen', 'green')

In [271]:
test = changer(test, 'pruple', 'purple')

In [274]:
test = changer(test,  'prurple', 'purple')

In [279]:
test = changer(test,  'puirple', 'purple')

In [281]:
test = changer(test,  'purlish', 'purpleish')

In [308]:
test = changer(test,  'yeellow', 'yellow')

In [310]:
test = changer(test, 'yeallow', 'yellow')

In [290]:
single(6748,  'ylow', 'yellow')

In [292]:
single(6191,  'yelowish', 'yellow')

In [294]:
single(1198,  'yelower', 'yellower')

In [296]:
test = changer(test,  'yellwo', 'yellow')

In [298]:
single(7918,  'yelloy', 'yellow')

In [300]:
single(8423,  'yellowy', 'yellow')

In [302]:
single(6963, 'yellowly', 'yellow')

In [304]:
single(5788, 'yelloish', 'yellowish')

In [306]:
single(12432,  'yelllow', 'yellow')

In [319]:
test = changer(test, 'turquise', 'turquoise' )

In [321]:
test = changer(test, 'turquiose', 'turquoise')

In [323]:
test = changer(test,  'turqouise', 'turquoise')

In [325]:
test = changer(test,  'turqoise', 'turquoise')

In [339]:
test = changer(test, 'bluish', 'blueish')

In [342]:
test = changer(test,  'bluey', 'blue')

In [345]:
test = changer(test,  'blueiosh', 'blueish')

In [347]:
test = changer(test, 'bluegreen', 'blue green')

In [353]:
test = changer(test,  'bluegrren', 'blue green')

In [ ]:
test[test['words'].str.contains( 'blu')]

In [362]:
processed_texts = test.words.values.tolist()

In [364]:
test.words.to_csv('data/colors/processed_texts.csv', index=False)

In [355]:
counts = Counter(' '.join(test.words.values.tolist()).split())
ones = [count for count in counts if counts[count] <= 1]

In [356]:
vocab = set([word for word in counts])
sorted(vocab)

['10',
 '2',
 '2nd',
 '6',
 '___',
 '_____',
 '____poppy',
 'a',
 'actual',
 'again',
 'ahaha',
 'ahh',
 'airline',
 'airport',
 'all',
 'almost',
 'amazing',
 'amethyst',
 'and',
 'answer',
 'any',
 'anytime',
 'apparel',
 'apple',
 'apply',
 'aqua',
 'arange',
 'are',
 'argh',
 'army',
 'ash',
 'avacado',
 'avocado',
 'aw',
 'aweome',
 'awesome',
 'b',
 'babby',
 'baby',
 'bad',
 'bag',
 'ballet',
 'banana',
 'barbie',
 'barf',
 'bark',
 'barkest',
 'barn',
 'barney',
 'barnie',
 'barnys',
 'based',
 'basic',
 'battleship',
 'bbright',
 'be',
 'beautiful',
 'beige',
 'berry',
 'better',
 'between',
 'bfight',
 'biege',
 'biggie',
 'bit',
 'bl',
 'black',
 'blackest',
 'blackish',
 'blah',
 'bland',
 'blandest',
 'ble',
 'blie',
 'bllue',
 'block',
 'blood',
 'blu',
 'blue',
 'blueish',
 'bluer',
 'blues',
 'bluest',
 'blunder',
 'blush',
 'bluw',
 'blye',
 'bodied',
 'bog',
 'bold',
 'boldest',
 'booger',
 'boring',
 'bornw',
 'borwn',
 'bownish',
 'box',
 'boy',
 'breakfast',
 'brei

['10',
 '2',
 '2nd',
 '6',
 '___',
 '_____',
 '____poppy',
 'a',
 'actual',
 'again',
 'ahaha',
 'ahh',
 'airline',
 'airport',
 'all',
 'almost',
 'amazing',
 'amethyst',
 'and',
 'answer',
 'any',
 'anytime',
 'apparel',
 'apple',
 'apply',
 'aqua',
 'arange',
 'are',
 'argh',
 'army',
 'ash',
 'avacado',
 'avocado',
 'aw',
 'aweome',
 'awesome',
 'b',
 'babby',
 'baby',
 'bad',
 'bag',
 'ballet',
 'banana',
 'barbie',
 'barf',
 'bark',
 'barkest',
 'barn',
 'barney',
 'barnie',
 'barnys',
 'based',
 'basic',
 'battleship',
 'bbright',
 'be',
 'beautiful',
 'beige',
 'berry',
 'better',
 'between',
 'bfight',
 'biege',
 'biggie',
 'bight',
 'birght',
 'birhgt',
 'birte',
 'bit',
 'bl',
 'black',
 'blackest',
 'blackish',
 'blah',
 'bland',
 'blandest',
 'ble',
 'blie',
 'bllue',
 'block',
 'blood',
 'blu',
 'blue',
 'bluegreen',
 'bluegrren',
 'blueiosh',
 'blueish',
 'bluer',
 'blues',
 'bluest',
 'bluey',
 'bluish',
 'blunder',
 'blush',
 'bluw',
 'blye',
 'bodied',
 'bog',
 'bold'

## Question 1: Improve the tokenizer [1 point]

This is the first required question – the first required modification to the default pipeline.

The function `tokenize_example` simply splits its string on whitespace and adds the required start and end symbols:

In [ ]:
'''
3) replacing tokens that appear once or
not at all in the training split with <unk>.
'''

In [15]:
#preprocessing functions for finding single-item words in training set
from collections import Counter
import re
ish_pattern = '\w+ish'
pattern = '\w+'

bow = []
for item in dev_texts_train + dev_texts_test:
    item = item.lower()
    words = re.findall(pattern, item)
    for word in words:
        bow.append(word)

#full vocab with counts
vocab = Counter(bow)
#sorted full vocab
sorted_vocab = sorted(vocab.items(), key=lambda x: x[1], reverse=True)
#ones values 
ones = {k:v for k, v in vocab.items() if v==1 }

In [123]:
greys = [
 'gray']

In [125]:
df = pd.DataFrame(dev_texts_train, columns=['raw'])
grays = list(df[df['raw'].str.contains('gray')].index)
grays

[]

In [118]:
for ind in grays:
    print(dev_texts_train[ind])
    dev_texts_train[ind] = dev_texts_train[ind].replace('gry', 'grey')
    print(dev_texts_train[ind])
#     print(dev_texts_train[ind].replace('gray', 'grey'))
    print('-'*100)

lighter gry
lighter grey
----------------------------------------------------------------------------------------------------


In [73]:
for gren in grens:
    dev_texts_train[gren] = dev_texts_train[gren].replace('gren', 'green')

In [43]:
dev_texts_train[4527] = 'grey green'

In [22]:
percent_ones = np.round(len(ones)/len(vocab), 3)
purples = [k for k in ones if k.startswith('p')]
yellows = [k for k in ones if k.startswith('y')]
blues = [k for k in ones if k.startswith('b')]

['barkest',
 'bog',
 'blah',
 'banana',
 'bownish',
 'bodied',
 'biggie',
 'bark',
 'blackish',
 'bueb',
 'burgandy',
 'blueiosh',
 'bornw',
 'blye',
 'blackest',
 'bfight',
 'bl',
 'barbie',
 'barn',
 'borwn',
 'brightet',
 'battleship',
 'beautiful',
 'briter',
 'bruight',
 'brickish',
 'based',
 'brwon',
 'brlue',
 'brigth',
 'bluegrren',
 'babby',
 'ble',
 'browning',
 'broiwn',
 'birhgt',
 'be',
 'berry',
 'brit',
 'breight',
 'brighttttt',
 'blues',
 'barf',
 'blie',
 'bluegreen',
 'bull',
 'brughtest',
 'bllue',
 'brughter',
 'bland',
 'birte',
 'brighness',
 'bronze',
 'but',
 'brigther',
 'brighet',
 'biege',
 'brightp',
 'barnys',
 'better',
 'birght',
 'bag',
 'brighly',
 'breakfast',
 'bubblegum',
 'bright5',
 'browny',
 'brught',
 'browin']

In [16]:
import re

def tokenize_example(s: str, ones: dict=ones, vocab: dict=vocab, suffs = ['er', 'est', 'ish']):
    '''
    Tokenizes string example and returns tokens with starting and ending tokens.
    Params: 
        s = string example
        ones = Predefined vocabulary dictionary of words appearing only once in corpus.
        suffs = Predefined suffix values that you would like to split on.
    '''
    
    unk = 'UNK_SYMBOL'
    suffixes = ['er', 'est', 'ish']
    
    #lowercase all words
    s = s.lower()
    
    #remove all punctuation
    words = re.findall('\w+', s)
    
    #replace instances of words appearing only once in vocab dictionary with UNK_SYMBOL
    words = [word.replace(word, unk) if word in ones or word not in vocab else word for word in words]
    final_words = []
    suffixes = tuple(suffs)
    
    for word in words:
        if word.endswith(suffixes):
            for suffix in suffixes:
                if word.endswith(suffix):
                    prefix = word.replace(suffix, '')
                    final_words.append(prefix)
                    final_words.append(suffix)
        else:
            final_words.append(word)
            
    return [START_SYMBOL] + final_words + [END_SYMBOL]

In [17]:
tokenize_example('brightest blue ### brighter than other nonsenseword dinosaur nonsensicalword')

['<s>',
 'bright',
 'est',
 'blue',
 'bright',
 'er',
 'than',
 'oth',
 'er',
 'UNK_SYMBOL',
 'UNK_SYMBOL',
 'UNK_SYMBOL',
 '</s>']

__Your task__: Modify `tokenize_example` so that it does something more sophisticated with the input text. 

__Notes__:

* There are useful ideas for this in [Monroe et al. 2017](https://transacl.org/ojs/index.php/tacl/article/view/1142)
* There is no requirement that you do word-level tokenization. Sub-word and multi-word are options.
* This question can interact with the size of your vocabulary (see just below), and in turn with decisions about how to use `UNK_SYMBOL`.

__Important__: don't forget to add the start and end symbols, else the resulting models will definitely be terrible! The following test will check that your tokenizer has this property:

In [18]:
def test_tokenize_example(func):
    s = "A test string"
    result = func(s)
    assert all(isinstance(tok, str) for tok in result), \
        "The tokenizer must return a list of strings."
    assert result[0] == START_SYMBOL, \
        "The tokenizer must add START_SYMBOL as the first token."
    assert result[-1] == END_SYMBOL, \
        "The tokenizer must add END_SYMBOL as the final token."

In [19]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_tokenize_example(tokenize_example)

## Use the tokenizer

Once the tokenizer is working, run the following cell to tokenize your inputs:

In [20]:
dev_seqs_train = [tokenize_example(s) for s in dev_texts_train]

dev_seqs_test = [tokenize_example(s) for s in dev_texts_test]

We use only the train set to derive a vocabulary for the model:

In [21]:
dev_vocab = sorted({w for toks in dev_seqs_train for w in toks})

dev_vocab += [UNK_SYMBOL]
len(dev_vocab)

482

In [22]:
#dev_vocab

It's important that the `UNK_SYMBOL` is included somewhere in this list. In test examples, words not seen in training will be mapped to `UNK_SYMBOL`. 

Conceptual note: If you model's vocab is the same as your train vocab, then `UNK_SYMBOL` will never be encountered during training, so it will be a random vector at test time.

In [23]:
len(dev_vocab), len(dev_seqs_train)

(482, 10417)

In [56]:
dev_vocab

['',
 '10',
 '2',
 '</s>',
 '<s>',
 'UNK_SYMBOL',
 '_____',
 'a',
 'again',
 'all',
 'almost',
 'and',
 'apple',
 'aqua',
 'are',
 'army',
 'avacado',
 'b',
 'baby',
 'bad',
 'ballet',
 'barney',
 'barnie',
 'basic',
 'bbright',
 'beige',
 'between',
 'bight',
 'bit',
 'black',
 'bland',
 'block',
 'blood',
 'blu',
 'blue',
 'bluey',
 'blund',
 'blush',
 'bluw',
 'bold',
 'boog',
 'boring',
 'box',
 'boy',
 'brick',
 'brigh',
 'bright',
 'brighte',
 'brightness',
 'brihgt',
 'briht',
 'brite',
 'brow',
 'browinsh',
 'brown',
 'brownis',
 'brt',
 'bue',
 'burnt',
 'button',
 'caca',
 'camo',
 'camoflage',
 'care',
 'caribbean',
 'catch',
 'cement',
 'chameleon',
 'choose',
 'citrus',
 'classic',
 'clay',
 'clear',
 'cloud',
 'cobalt',
 'color',
 'colored',
 'concrete',
 'cone',
 'confederate',
 'cool',
 'coral',
 'corn',
 'cornflow',
 'crap',
 'cyan',
 'd',
 'dark',
 'darke',
 'day',
 'deep',
 'definitely',
 'different',
 'dim',
 'dimm',
 'dingy',
 'dirty',
 'dk',
 'do',
 'doesn',
 'doi

## Question 2: Improve the color representations [1 point]

This is the second required pipeline improvement for the assignment. 

The following functions do nothing at all to the raw input colors we get from the corpus. 

In [62]:
RANGES_RGB = (256.0, 256.0, 256.0)
RANGES_HSV = (361.0, 201.0, 201.0)
C_EPSILON = 1e-4
    
class ColorVectorizer(object):
    
    def vectorize_all(self, colors, hsv=None):
        '''
        :param colors: A sequence of length-3 vectors or 1D array-like objects containing
                      RGB coordinates in the range [0, 256).
        :param bool hsv: If `True`, input is assumed to be in HSV space in the range
                         [0, 360], [0, 100], [0, 100]; if `False`, input should be in RGB
                         space in the range [0, 256). `None` (default) means take the
                         color space from the value given to the constructor.
        :return np.ndarray: An array of the vectorized form of each color in `colors`
                            (first dimension is the index of the color in the `colors`).
        >>> BucketsVectorizer((2, 2, 2)).vectorize_all([(0, 0, 0), (255, 0, 0)])
        array([0, 4], dtype=int32)
        '''
        return np.array([self.vectorize(c, hsv=hsv) for c in colors])
    

class FourierVectorizer(ColorVectorizer):
    '''
    Vectorizes colors by converting them to a truncated frequency representation.
    This vectorizer can only vectorize, not unvectorize.
    '''
    def __init__(self, resolution, hsv=False):
        '''
        :param resolution: The number of dimensions to truncate the frequency
                           representation (the vectorized representation will be
                           *twice* this, because the frequency representation uses
                           complex numbers). Should be an even number between 0 and
                           the range of each internal color space dimension, or a
                           length-3 sequence of such numbers.
                           
        :param bool hsv: If `True`, the internal color space used by the vectorizer
                         will be HSV. Input and output color spaces can be configured
                         on a per-call basis by using the `hsv` parameter of
                         `vectorize` and `unvectorize`.
        '''
        if len(resolution) == 1:
            resolution = resolution * 3
        self.resolution = resolution
        self.output_size = np.prod(resolution) * 2
        self.hsv = hsv

    def vectorize(self, color, hsv=None):
        '''
        :param color: An length-3 vector or 1D array-like object containing
                      color coordinates.
        :param bool hsv: If `True`, input is assumed to be in HSV space in the range
                         [0, 360], [0, 100], [0, 100]; if `False`, input should be in RGB
                         space in the range [0, 255]. `None` (default) means take the
                         color space from the value given to the constructor.
        :return np.ndarray: The color in the Fourier representation,
                            a vector of shape `(prod(resolution) * 2,)`.
        >>> normalize = lambda v: np.where(v.round(2) == 0.0, 0.0, v.round(2))
        >>> normalize(FourierVectorizer([2]).vectorize((255, 0, 0)))
        array([ 1.,  1.,  1.,  1., -1., -1., -1., -1.,  0.,  0.,  0.,  0.,  0.,
                0.,  0.,  0.], dtype=float32)
        >>> normalize(FourierVectorizer([2]).vectorize((180, 100, 100), hsv=True))
        array([ 1., -1., -1.,  1.,  1., -1., -1.,  1.,  0.,  0.,  0.,  0.,  0.,
                0.,  0.,  0.], dtype=float32)
        >>> normalize(FourierVectorizer([2], hsv=True).vectorize((0, 100, 100)))
        array([ 1., -1., -1.,  1.,  1., -1., -1.,  1.,  0.,  0.,  0.,  0.,  0.,
                0.,  0.,  0.], dtype=float32)
        >>> normalize(FourierVectorizer([2], hsv=True).vectorize((0, 255, 255), hsv=False))
        array([ 1., -1., -1.,  1., -1.,  1.,  1., -1.,  0.,  0.,  0.,  0.,  0.,
                0.,  0.,  0.], dtype=float32)
        '''
        return self.vectorize_all([color], hsv=hsv)[0]

    def vectorize_all(self, colors, hsv=None):
        '''
        >>> normalize = lambda v: np.where(v.round(2) == 0.0, 0.0, v.round(2))
        >>> normalize(FourierVectorizer([2]).vectorize_all([(255, 0, 0), (0, 255, 255)]))
        array([[ 1.,  1.,  1.,  1., -1., -1., -1., -1.,  0.,  0.,  0.,  0.,  0.,
                 0.,  0.,  0.],
               [ 1., -1., -1.,  1.,  1., -1., -1.,  1.,  0.,  0.,  0.,  0.,  0.,
                 0.,  0.,  0.]], dtype=float32)
        '''
        if hsv is None:
            hsv = self.hsv

        colors = np.array([colors])
        assert len(colors.shape) == 3, colors.shape
        assert colors.shape[2] == 3, colors.shape

        ranges = np.array(RANGES_HSV if self.hsv else RANGES_RGB)
        if hsv and not self.hsv:
            c_hsv = colors
            color_0_1 = skimage.color.hsv2rgb(c_hsv / (np.array(RANGES_HSV) - 1.0))
        elif not hsv and self.hsv:
            c_rgb = colors
            color_0_1 = skimage.color.rgb2hsv(c_rgb / (np.array(RANGES_RGB) - 1.0))
        else:
            color_0_1 = colors / (ranges - 1.0)

        # Using a Fourier representation causes colors at the boundary of the
        # space to behave as if the space is toroidal: red = 255 would be
        # about the same as red = 0. We don't want this...
        xyz = color_0_1[0] / 2.0
        if self.hsv:
            # ...*except* in the case of HSV: H is in fact a polar coordinate.
            xyz[:, 0] *= 2.0

        # ax, ay, az = [np.hstack([np.arange(0, g / 2), np.arange(r - g / 2, r)])
        #               for g, r in zip(self.resolution, ranges)]
        ax, ay, az = [np.arange(0, g) for g, r in zip(self.resolution, ranges)]
        gx, gy, gz = np.meshgrid(ax, ay, az)

        arg = (np.multiply.outer(xyz[:, 0], gx) +
               np.multiply.outer(xyz[:, 1], gy) +
               np.multiply.outer(xyz[:, 2], gz))
        assert arg.shape == (xyz.shape[0],) + tuple(self.resolution), arg.shape
        repr_complex = np.exp(-2j * np.pi * (arg % 1.0)).swapaxes(1, 2).reshape((xyz.shape[0], -1))
        result = np.hstack([repr_complex.real, repr_complex.imag]).astype(np.float32)
        return result


In [63]:
def represent_color_context(colors):
    return [represent_color(color) for color in colors]


def represent_color(color: list) -> np.array:
    '''
    Given an input list of floats in HSL format, converts input
    into a numpy array in HSV format.
    '''
    
    f = FourierVectorizer([3],hsv=True)
    
    #convert HLS to HSV
    arr = np.array(color)
    hue = arr[0]
    sat = arr[1]
    lum = arr[2]
    value = lum + sat * min(lum, 1 - lum)
    arr[2] = value
    Sv = 0 if value == 0 else 2*(1-(lum/value))
    arr[1] = Sv
    
    transform = f.vectorize(arr, hsv=True)
    transform = [float(x) for x in transform]
    return transform

The following test seeks to ensure only that the output of your `represent_color_context` will be compatible with the models we are creating:

In [64]:
def test_represent_color_context(func):
    """`func` should be `represent_color_context`"""
    example = [
        [0.786, 0.58, 0.87],
        [0.689, 0.44, 0.92],
        [0.628, 0.32, 0.81]]
    result = func(example)
    assert len(result) == len(example), \
        ("Color context representations need to represent each color "
         "separately. (We assume the final color is the target.)")
    for i, color in enumerate(result):
        assert all(isinstance(x, float) for x in color), \
            ("All color representations should be lists of floats. "
             "Color {} is {}".format(i, color))

In [65]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_represent_color_context(represent_color_context)

## Use the color representer

The following cell just runs your `represent_color_context` on the train and test sets:

In [66]:
dev_cols_train = [represent_color_context(colors) for colors in dev_rawcols_train]

dev_cols_test = [represent_color_context(colors) for colors in dev_rawcols_test]

list(map(len, [dev_cols_train, dev_cols_test]))

[10417, 3473]

At this point, our preprocessing steps are complete, and we can fit a first model.

## Initial model

The first model is configured right now to be a small model run for just a few iterations. It should be enough to get traction, but it's unlikely to be a great model. You are free to modify this configuration if you wish; it is here just for demonstration and testing:

In [67]:
dev_mod = ContextualColorDescriber(
    dev_vocab,
    early_stopping=True)

In [68]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    %time _ = dev_mod.fit(dev_cols_train, dev_seqs_train)
else:
    dev_mod.fit(dev_cols_train, dev_seqs_train)

Stopping after epoch 28. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 36.19747829437256

CPU times: user 1min 46s, sys: 41.1 s, total: 2min 27s
Wall time: 1min 15s


The canonical bake-off evaluation function is `evaluate`. Our primary metric is `listener_accuracy`; the BLEU score is included as a check to ensure that your system is speaking English!

In [69]:
evaluation = dev_mod.evaluate(dev_cols_test, dev_seqs_test)

In [70]:
evaluation.keys()

dict_keys(['listener_accuracy', 'corpus_bleu', 'target_index', 'predicted_index', 'predicted_utterance'])

In [71]:
evaluation['listener_accuracy']

0.3648142816009214

In [72]:
dev_mod.listener_accuracy(dev_cols_test, dev_seqs_test)

0.3648142816009214

In [73]:
evaluation['corpus_bleu']

0.561595912984063

In [74]:
bleu, predicted_utterances = dev_mod.corpus_bleu(dev_cols_test, dev_seqs_test)

bleu

0.561595912984063

In [75]:
evaluation['target_index'][: 5]

[2, 2, 2, 2, 2]

In [76]:
evaluation['predicted_index'][: 5]

[1, 1, 2, 2, 0]

In [77]:
evaluation['predicted_utterance'][:5]

[['<s>', 'bright', 'green', '</s>'],
 ['<s>', 'bright', 'green', '</s>'],
 ['<s>', 'bright', 'green', '</s>'],
 ['<s>', 'bright', 'green', '</s>'],
 ['<s>', 'bright', 'green', '</s>']]

We can also see the model's predicted sequences given color context inputs:

In [78]:
dev_mod.predict(dev_cols_test[: 1])

[['<s>', 'bright', 'green', '</s>']]

In [79]:
dev_seqs_test[: 1]

[['<s>', 'green', 'ish', 'brown', '</s>']]

## Question 3: GloVe embeddings [1 point]

The above model uses a random initial embedding, as configured by the decoder used by `ContextualColorDescriber`. This homework question asks you to consider using GloVe inputs. 

__Your task__: Complete `create_glove_embedding` so that it creates a GloVe embedding based on your model vocabulary. This isn't mean to be analytically challenging, but rather just to create a basis for you to try out other kinds of rich initialization.

In [92]:
GLOVE_HOME = os.path.join('data/', 'glove.6B')

In [93]:
def create_glove_embedding(vocab, glove_base_filename='glove.6B.50d.txt'):

    # Use `utils.glove2dict` to read in the GloVe file:
    path = os.path.join(GLOVE_HOME, glove_base_filename)
    glove_embeddings = utils.glove2dict(path)
    

    # Use `utils.create_pretrained_embedding` to create the embedding.
    # This function will, by default, ensure that START_TOKEN,
    # END_TOKEN, and UNK_TOKEN are included in the embedding.
    embed = utils.create_pretrained_embedding(glove_embeddings, vocab)


    # Be sure to return the embedding you create as well as the
    # vocabulary returned by `utils.create_pretrained_embedding`,
    # which is likely to have been modified from the input `vocab`.
    
    return embed[0], embed[1]


In [82]:
data = create_glove_embedding(dev_vocab)

In [83]:
d = {v:k for k,v in zip(data[0], data[1])}

In [94]:
def test_create_glove_embedding(func):
    vocab = ['NLU', 'is', 'the', 'future', '.', '$UNK', '<s>', '</s>']
    glove_embedding, glove_vocab = func(vocab, 'glove.6B.50d.txt')
    assert isinstance(glove_embedding, np.ndarray), \
        "Expected embedding type {}; got {}".format(
        glove_embedding.__class__.__name__, glove_embedding.__class__.__name__)
    assert glove_embedding.shape == (8, 50), \
        "Expected embedding shape (8, 50); got {}".format(glove_embedding.shape)
    assert glove_vocab == vocab, \
        "Expected vocab {}; got {}".format(vocab, glove_vocab)

In [95]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_create_glove_embedding(create_glove_embedding)

## Try the GloVe representations

The extent to which GloVe is useful will depend heavily on how aligned your tokenization scheme is with the GloVe vocabulary. For example, if you did character-level tokenization, then the GloVe embedding space is not well-aligned with your tokenizer and using GloVe should have little no positive effect.

Let's see if GloVe helped for our development data:

In [96]:
dev_glove_embedding, dev_glove_vocab = create_glove_embedding(dev_vocab, 'glove.6B.300d.txt')

In [97]:
len(dev_vocab)

482

In [98]:
len(dev_glove_vocab)

482

In [99]:
dev_mod_glove = ContextualColorDescriber(
    dev_glove_vocab,
    embedding=dev_glove_embedding,
    early_stopping=True)

In [100]:
_ = dev_mod_glove.fit(dev_cols_train, dev_seqs_train)

Stopping after epoch 12. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 39.05716943740845

In [101]:
dev_mod_glove.listener_accuracy(dev_cols_test, dev_seqs_test)

0.36337460408868416

You probably saw a small boost, assuming your tokeization scheme leads to good overlap with the GloVe vocabulary. The input representations are larger than in our previous model (at least as I configured things), so we would need to do more runs with higher `max_iter` values to see whether this is worthwhile overall.

## Question 4: Color context [3 points]

The final required homework question is the most challenging, but it should set you up to think in much more flexible ways about the underlying model we're using.

The question asks you to modify various model components in `torch_color_describer.py`. The section called [Modifying the core model](colors_overview.ipynb#Modifying-the-core-model) from the core unit notebook provides a number of examples illustrating the basic techniques, so you might review that material if you get stuck here.

__Your task__: Building on ideas from [Monroe et al. 2017](https://transacl.org/ojs/index.php/tacl/article/view/1142), you will redesign the model so that the target color (the final one in the context) is appended to each input token that gets processed by the decoder. The question asks you to subclass the `Decoder` and `EncoderDecoder` from `torch_color_describer.py` so that you can build models that do this.

__Step 1__: Modify the `Decoder` so that the input vector to the model at each timestep is not just a token representation `x` but the concatenation of `x` with the representation of the target color.

__Notes__:

* You might notice at this point that the original `Decoder.forward` method has an optional keyword argument `target_colors` that is passed to `Decoder.get_embeddings`. Because this is already in place, all you have to do is modify the `get_embeddings` method to use this argument.

* The change affects the configuration of `self.rnn`, so you need to subclass the `__init__` method as well, so that its `input_size` argument accomodates the embedding as well as the color representations.

* You can do the relevant operations efficiently in pure PyTorch using `repeat_interleave` and `cat`, but the important thing is to get a working implementation – you can always optimize the code later if the ideas prove useful to you. 

Here's skeleton code for you to flesh out:

In [ ]:
from torch_color_describer import Decoder
import torch
import torch.nn as nn


class ColorContextDecoder(Decoder):
    def __init__(self, color_dim, *args, **kwargs):
        self.color_dim = color_dim
        super().__init__(*args, **kwargs)

        # Fix the `self.rnn` attribute:
        ##### YOUR CODE HERE


    def get_embeddings(self, word_seqs, target_colors=None):
        """
        You can assume that `target_colors` is a tensor of shape
        (m, n), where m is the length of the batch (same as
        `word_seqs.shape[0]`) and n is the dimensionality of the
        color representations the model is using. The goal is
        to attached each color vector i to each of the tokens in
        the ith sequence of (the embedded version of) `word_seqs`.

        """
        ##### YOUR CODE HERE



Step 1 is the most demanding of the steps in terms of tensor wrangling. It's important to have a clear idea of what you are trying to achieve and to unit test `get_embeddings` so that you can check that it has realized your vision. The following test should help with that:

In [ ]:
def test_get_embeddings(decoder_class):
    """
    It's assumed that the input to this will be `ColorContextDecoder`.
    You pass in the class, and the function initalizes it with the test
    parameters.
    """
    dec = decoder_class(
        color_dim=3,   # For these, we mainly want *different*
        vocab_size=10, # dimensions so that we reliably get
        embed_dim=4,   # dimensionality errors if something
        hidden_dim=5)  # isn't working.

    # This step just changes the embedding to one with values
    # that are easy to inspect and definitely will not change
    # between runs:
    dec.embedding = nn.Embedding.from_pretrained(
        torch.FloatTensor([
            [10, 11, 12, 13],
            [14, 15, 16, 17],
            [18, 19, 20, 21]]))

    # These are the incoming sequences -- lists of indices
    # into the rows of `dec.embedding`:
    word_seqs = torch.tensor([
        [0,1,2],
        [2,0,1]])

    # Target colors as small floats that will be easy to track:
    target_colors = torch.tensor([
        [0.1, 0.2, 0.3],
        [0.7, 0.8, 0.9]])

    # The desired return value: one list of tensors for each of
    # the two sequences in `word_seqs`. Each index is replaced
    # with its vector from `dec.embedding` and has the
    # corrresponding color from `target_colors` appended to it.
    expected = torch.tensor([
        [[10., 11., 12., 13.,  0.1,  0.2,  0.3],
         [14., 15., 16., 17.,  0.1,  0.2,  0.3],
         [18., 19., 20., 21.,  0.1,  0.2,  0.3]],

        [[18., 19., 20., 21.,  0.7,  0.8,  0.9],
         [10., 11., 12., 13.,  0.7,  0.8,  0.9],
         [14., 15., 16., 17.,  0.7,  0.8,  0.9]]])

    result = dec.get_embeddings(word_seqs, target_colors=target_colors)

    assert expected.shape == result.shape, \
        "Expected shape {}; got shape {}".format(expected.shape, result.shape)

    assert torch.all(expected.eq(result)), \
        ("Your result has the desired shape but the values aren't correct. "
         "Here's what your function creates; compare it with `expected` "
         "from the test:\n{}".format(result))

In [ ]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_get_embeddings(ColorContextDecoder)

__Step 2__: Modify the `EncoderDecoder`. For this, you just need to make a small change to the `forward` method: extract the target colors from `color_seqs` and feed them to the decoder.

In [ ]:
from torch_color_describer import EncoderDecoder

class ColorizedEncoderDecoder(EncoderDecoder):

    def forward(self,
            color_seqs,
            word_seqs,
            seq_lengths=None,
            hidden=None,
            targets=None):
        if hidden is None:
            hidden = self.encoder(color_seqs)

        # Extract the target colors from `color_seqs` and
        # feed them to the decoder, which already has a
        # `target_colors` keyword.

        ##### YOUR CODE HERE



        # Your decoder will return `output, hidden` pairs; the
        # following will handle the two return situations that
        # the code needs to consider -- training and prediction.
        if self.training:
            return output
        else:
            return output, hidden

__Step 3__: Finally, as in the examples in [Modifying the core model](colors_overview.ipynb#Modifying-the-core-model), you need to modify the `build_graph` method of `ContextualColorDescriber` so that it uses your new `ColorContextDecoder` and `ColorizedEncoderDecoder`. Here's starter code:

In [ ]:
from torch_color_describer import Encoder

class ColorizedInputDescriber(ContextualColorDescriber):

    def build_graph(self):

        # We didn't modify the encoder, so this is
        # just copied over from the original:
        encoder = Encoder(
            color_dim=self.color_dim,
            hidden_dim=self.hidden_dim)

        # Use your `ColorContextDecoder`, making sure
        # to pass in all the keyword arguments coming
        # from `ColorizedInputDescriber`:

        ##### YOUR CODE HERE



        # Return a `ColorizedEncoderDecoder` that uses
        # your encoder and decoder:

        ##### YOUR CODE HERE



That's it! Since these modifications are pretty intricate, you might want to use [a toy dataset](colors_overview.ipynb#Toy-problems-for-development-work) to debug it:

In [ ]:
def test_full_system(describer_class):
    toy_color_seqs, toy_word_seqs, toy_vocab = create_example_dataset(
        group_size=50, vec_dim=2)

    toy_color_seqs_train, toy_color_seqs_test, toy_word_seqs_train, toy_word_seqs_test = \
        train_test_split(toy_color_seqs, toy_word_seqs)

    toy_mod = describer_class(toy_vocab)

    _ = toy_mod.fit(toy_color_seqs_train, toy_word_seqs_train)

    acc = toy_mod.listener_accuracy(toy_color_seqs_test, toy_word_seqs_test)

    return acc

In [ ]:
test_full_system(ColorizedInputDescriber)

If that worked, then you can now try this model on SCC problems!

## Your original system [3 points]

There are many options for your original system, which consists of the full pipeline – all preprocessing and modeling steps. You are free to use any model you like, as long as you subclass `ContextualColorDescriber` in a way that allows its `evaluate` method to behave in the expected way.

So that we can evaluate models in a uniform way for the bake-off, we ask that you modify the function `evaluate_original_system` below so that it accepts a trained instance of your model and does any preprocessing steps required by your model.

If we seek to reproduce your results, we will rerun this entire notebook. Thus, it is fine if your `evaluate_original_system` makes use of functions you wrote or modified above this cell.

In [ ]:
def evaluate_original_system(trained_model, color_seqs_test, texts_test):
    """
    Feel free to modify this code to accommodate the needs of
    your system. Just keep in mind that it will get raw corpus
    examples as inputs for the bake-off.

    """
    # `word_seqs_test` is a list of strings, so tokenize each of
    # its elements:
    tok_seqs = [tokenize_example(s) for s in texts_test]

    col_seqs = [represent_color_context(colors)
                for colors in color_seqs_test]


    # Optionally include other preprocessing steps here. Note:
    # DO NOT RETRAIN YOUR MODEL AS PART OF THIS EVALUATION!
    # It's a tempting step, but it's a mistake and will get
    # you disqualified!

    # The following core score calculations are required:
    evaluation = trained_model.evaluate(col_seqs, tok_seqs)

    return evaluation

If `evaluate_original_system` works on test sets you create from the corpus distribution, then it will work for the bake-off, so consider checking that. For example, this would check that `dev_mod` above passes muster:

In [ ]:
my_evaluation = evaluate_original_system(dev_mod, dev_rawcols_test, dev_texts_test)

In [ ]:
my_evaluation['listener_accuracy']

In [ ]:
my_evaluation['corpus_bleu']

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies. We also ask that you report the best **listener_accuracy** score your system got during development, just to help us understand how systems performed overall.

<font color='red'>Please review the descriptions in the following comment and follow the instructions.</font>

In [ ]:
# PLEASE MAKE SURE TO INCLUDE THE FOLLOWING BETWEEN THE START AND STOP COMMENTS:
#   1) Textual description of your system.
#   2) The code for your original system.
#   3) The score achieved by your system in place of MY_NUMBER.
#        With no other changes to that line.
#        You should report your score as a decimal value <=1.0
# PLEASE MAKE SURE NOT TO DELETE OR EDIT THE START AND STOP COMMENTS

# NOTE: MODULES, CODE AND DATASETS REQUIRED FOR YOUR ORIGINAL SYSTEM
# SHOULD BE ADDED BELOW THE 'IS_GRADESCOPE_ENV' CHECK CONDITION. DOING
# SO ABOVE THE CHECK MAY CAUSE THE AUTOGRADER TO FAIL.

# START COMMENT: Enter your system description in this cell.
# My peak score was: MY_NUMBER
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass

# STOP COMMENT: Please do not remove this comment.

## Bakeoff [1 point]

For the bake-off, we will use our original test set. The function you need to run for the submission is the following, which uses your `evaluate_original_system` from above:

In [ ]:
def create_bakeoff_submission(
        trained_model,
        output_filename='cs224u-colors-bakeoff-entry.csv'):
    bakeoff_src_filename = os.path.join(
        "data", "colors", "cs224u-colors-test.csv")

    bakeoff_corpus = ColorsCorpusReader(bakeoff_src_filename)

    # This code just extracts the colors and texts from the new corpus:
    bakeoff_rawcols, bakeoff_texts = zip(*[
        [ex.colors, ex.contents] for ex in bakeoff_corpus.read()])

    # Original system function call; `trained_model` is your trained model:
    evaluation = evaluate_original_system(
        trained_model, bakeoff_rawcols, bakeoff_texts)

    evaluation['bakeoff_text'] = bakeoff_texts

    df = pd.DataFrame(evaluation)
    df.to_csv(output_filename)

In [ ]:
# This check ensure that the following code only runs on the local environment only.
# The following call will not be run on the autograder environment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass
    create_bakeoff_submission(dev_mod)

This creates a file `cs224u-colors-bakeoff-entry.csv` in the current directory. That file should be uploaded as-is. Please do not change its name.

Only one upload per team is permitted, and you should do no tuning of your system based on what you see in the file – you should not study that file in anyway, beyond perhaps checking that it contains what you expected it to contain. The upload function will do some additional checking to ensure that your file is well-formed.

The nature of our evaluation is such that we have to release the full test set with all labels. Thus, we have to trust you not to make any use of the test set during development. Recall:

1. Only one evaluation is permitted.
1. No additional system tuning is permitted once the bake-off has started.

Systems will be ranked primarily by `listener_accuracy`, but we will also consider their `corpus_bleu` scores. However, the BLEU score is just a simple check that your system is speaking some version of English that corresponds in some meaningful way to the gold descriptions, so you should concentrate on `listener_accuracy`.

People who enter will receive the additional homework point, and people whose systems achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points.

## Submission Instruction

Review and follow the [Homework and bake-off code: Formatting guide](hw_formatting_guide.ipynb).
Please do not change the file name as described below.

Submit the following files to Gradescope:

- `hw_colors.ipynb` (this notebook)
- `cs224u-colors-bakeoff-entry.csv` (bake-off output)